In [1]:
import numpy as np
import numpy.ma as ma
import pandas as pd
import datetime

import matplotlib.pyplot as plt

from tqdm import tqdm

import feather

from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

import os
os.environ['KMP_DUPLICATE_LIB_OK']='True'

import xgboost as xgb

import keras
from keras.preprocessing import sequence
from keras import layers, Input, Model
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense, Lambda

from sklearn import preprocessing



Using TensorFlow backend.


In [2]:
# Load in the time_series from the previous stage in the pipeline
correlations = feather.read_dataframe("../data_files/B/once/35days/10mins/best_correlations_offset_10mins_code_hour_day.feather")
correlations = correlations.set_index(correlations.columns[0])


In [3]:
# time_series = feather.read_dataframe("../data_files/B/once/35days/10mins/diff_percent_from_code_hour_day_-_full_segment_time_series.feather")
# time_series = time_series.set_index(time_series.columns[0])

# time_series = time_series.rename(renamer, axis="columns")

In [4]:
stop_events = feather.read_dataframe("../data_files/B/once/35days/stop_events_with_geo_train_test_averages.feather")
stop_events = stop_events.set_index(stop_events.columns[0])

In [5]:
stop_events["arrival_10mins"] = stop_events["actualArrival"].dt.round("10min")
stop_events['offset_timestamp'] = stop_events["arrival_10mins"] - pd.Timedelta("10 minute")

In [6]:
# We need to generate this from scratch as we need both test and train data. 
time_series = stop_events.pivot_table(
        index="arrival_10mins",
        columns="segment_code",
        values="diff_percent_full_segment_and_median_by_segment_code_and_hour_and_day",
        aggfunc=np.mean,
    )

In [7]:
# For each stop event we want to add columns with the values of the averages for the n best correlators ofset by 10 mins. 
# Steps to do this: 
# Make a column with the offset time stamp
# For each segment_code merge in the correlations
# Find a way to merge in the time series values based on the segment_code, offset_timestamp and correlation_segemnt (might need a pivot?)


In [8]:
stop_events = stop_events[['date', 'stopCode', 'aimedArrival', 'aimedDeparture', 'actualArrival', 'actualDeparture', 'prev_stopCode', 'prev_aimedArrival',
       'prev_aimedDeparture', 'prev_actualArrival', 'prev_actualDeparture',
       'timingPoint', 'prev_timingPoint', 'segment_code', 'segment_name', 'line_distance', 'to_centre_dist', 'direction_degrees', 'train', 'test','full_duration','median_full_durations_by_segment_code',
       'median_full_durations_by_segment_code_and_hour',
       'median_full_durations_by_segment_code_and_hour_and_day',
       'diff_full_segment_and_median_by_segment_code',
       'diff_full_segment_and_median_by_segment_code_and_hour_and_day',
       'diff_percent_full_segment_and_median_by_segment_code',
       'diff_percent_full_segment_and_median_by_segment_code_and_hour_and_day',
       'offset_timestamp']]


In [9]:
stop_events = stop_events.merge(correlations, left_on="segment_code", right_index=True)

In [10]:
def best(row):
    try:
        value = time_series.loc[row[0],row[1]] 
    except: 
#         print("--------------")
#         print(row.name)
#         print(row[0])
#         print(row[1])
        value = np.nan
    return value

In [11]:
stop_events['best_0'] = stop_events[['offset_timestamp', '0']].apply(best, axis=1)

In [ ]:
stop_events = stop_events.dropna(subset=['best_0'])

In [12]:
stop_events['best_1'] = stop_events[['offset_timestamp', '1']].apply(best, axis=1)
stop_events['best_2'] = stop_events[['offset_timestamp', '2']].apply(best, axis=1)
stop_events['best_3'] = stop_events[['offset_timestamp', '3']].apply(best, axis=1)
stop_events['best_4'] = stop_events[['offset_timestamp', '4']].apply(best, axis=1)

In [28]:
stop_events['best_5'] = stop_events[['offset_timestamp', '5']].apply(best, axis=1)
stop_events['best_6'] = stop_events[['offset_timestamp', '6']].apply(best, axis=1)
stop_events['best_7'] = stop_events[['offset_timestamp', '7']].apply(best, axis=1)
stop_events['best_8'] = stop_events[['offset_timestamp', '8']].apply(best, axis=1)
stop_events['best_9'] = stop_events[['offset_timestamp', '9']].apply(best, axis=1)

In [31]:
stop_events = stop_events.replace([np.nan, -np.inf], 0)

In [32]:
train_matrix = stop_events[stop_events['train']][['line_distance', 'to_centre_dist', 'direction_degrees', 'best_0', 'best_1', 'best_2', 'best_3', 'best_4', 'best_5', 'best_6', 'best_7', 'best_8', 'best_9', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']].values
train_target = stop_events[stop_events['train']]['diff_percent_full_segment_and_median_by_segment_code_and_hour_and_day'].values

test_matrix = stop_events[stop_events['test']][['line_distance', 'to_centre_dist', 'direction_degrees', 'best_0', 'best_1', 'best_2', 'best_3', 'best_4', 'best_5', 'best_6', 'best_7', 'best_8', 'best_9', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39']].values
test_target = stop_events[stop_events['test']]['diff_percent_full_segment_and_median_by_segment_code_and_hour_and_day'].values



In [15]:
stop_events.head(20)

,date,stopCode,aimedArrival,aimedDeparture,actualArrival,actualDeparture,prev_stopCode,prev_aimedArrival,prev_aimedDeparture,prev_actualArrival,...,55,56,57,58,59,best_0,best_1,best_2,best_3,best_4
index,,,,,,,,,,,,,,,,,,,,,
0,2018-09-18,1200BOB20156,2018-09-18 07:03:00,2018-09-18 07:03:00,2018-09-18 07:02:25,2018-09-18 07:02:30,1200BOB20158,2018-09-18 07:02:00,2018-09-18 07:02:00,2018-09-18 07:02:03,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
78,2018-09-18,1200BOB20156,2018-09-18 16:07:00,2018-09-18 16:07:00,2018-09-18 16:08:12,2018-09-18 16:08:15,1200BOB20158,2018-09-18 16:06:00,2018-09-18 16:06:00,2018-09-18 16:06:58,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,-8.77193,0.000000,0.0
154,2018-09-18,1200BOB20156,2018-09-18 17:57:00,2018-09-18 17:57:00,2018-09-18 17:57:13,2018-09-18 17:57:18,1200BOB20158,2018-09-18 17:56:00,2018-09-18 17:56:00,2018-09-18 17:56:44,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
220,2018-09-18,1200BOB20156,2018-09-18 08:47:00,2018-09-18 08:47:00,2018-09-18 08:46:08,2018-09-18 08:46:12,1200BOB20158,2018-09-18 08:46:00,2018-09-18 08:46:00,2018-09-18 08:45:48,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
298,2018-09-18,1200BOB20156,2018-09-18 10:37:00,2018-09-18 10:37:00,2018-09-18 10:36:53,2018-09-18 10:37:21,1200BOB20158,2018-09-18 10:36:00,2018-09-18 10:36:00,2018-09-18 10:36:05,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
376,2018-09-18,1200BOB20156,2018-09-18 12:27:00,2018-09-18 12:27:00,2018-09-18 12:26:38,2018-09-18 12:26:43,1200BOB20158,2018-09-18 12:26:00,2018-09-18 12:26:00,2018-09-18 12:25:20,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
451,2018-09-18,1200BOB20156,2018-09-18 14:17:00,2018-09-18 14:17:00,2018-09-18 14:17:41,2018-09-18 14:17:44,1200BOB20158,2018-09-18 14:16:00,2018-09-18 14:16:00,2018-09-18 14:16:36,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0
491,2018-09-18,1200BOB20156,2018-09-18 07:37:00,2018-09-18 07:37:00,2018-09-18 07:40:32,2018-09-18 07:40:33,1200BOB20158,2018-09-18 07:36:00,2018-09-18 07:36:00,2018-09-18 07:40:21,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,-4.545455,0.0
551,2018-09-18,1200BOB20156,2018-09-18 16:47:00,2018-09-18 16:47:00,2018-09-18 16:47:57,2018-09-18 16:47:59,1200BOB20158,2018-09-18 16:46:00,2018-09-18 16:46:00,2018-09-18 16:47:05,...,0.225653,0.222496,0.196888,0.196597,0.195317,0.0,0.0,0.00000,0.000000,0.0


In [33]:
reg = LinearRegression().fit(train_matrix, train_target)


In [34]:
reg.score(test_matrix, test_target)



-0.0005597516528617064

In [35]:
rmse_lr = np.sqrt(mean_squared_error(test_target, reg.predict(test_matrix)))


In [36]:
print(rmse_lr)


87.23064655428036


In [37]:
data_dmatrix = xgb.DMatrix(data=train_matrix,label=train_target)


In [38]:
xg_reg = xgb.XGBRegressor(objective ='reg:linear', booster='gblinear', max_depth=15)

In [39]:
xg_reg.fit(train_matrix, train_target)


XGBRegressor(base_score=0.5, booster='gblinear', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, importance_type='gain',
       learning_rate=0.1, max_delta_step=0, max_depth=15,
       min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
       nthread=None, objective='reg:linear', random_state=0, reg_alpha=0,
       reg_lambda=1, scale_pos_weight=1, seed=None, silent=True,
       subsample=1)

In [40]:
rmse_xg = np.sqrt(mean_squared_error(test_target, xg_reg.predict(test_matrix)))


In [41]:
print(rmse_xg)

87.4274008307711


In [46]:
scaler_matrix = preprocessing.StandardScaler().fit(train_matrix)

train_matrix_scaled = scaler_matrix.transform(train_matrix)
test_matrix_scaled = scaler_matrix.transform(test_matrix)

scaler_target = preprocessing.StandardScaler().fit(train_target[:,None])

train_target_scaled = scaler_target.transform(train_target[:,None])
test_target_scaled = scaler_target.transform(test_target[:,None])




In [47]:
def create_fully_connected(input_width):
    model = Sequential()
    
    model.add(Dense(32, input_dim=input_width, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(32, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(12, activation='relu'))
    model.add(layers.Dropout(0.2))
    model.add(Dense(1, activation='tanh'))
    model.summary()
    
    return model

In [49]:
model = create_fully_connected(train_matrix.shape[1])

callbacks_list = [
    keras.callbacks.EarlyStopping(monitor='val_loss', patience=2),
    keras.callbacks.ModelCheckpoint(filepath='full_conn_model.h5', monitor='val_loss', save_best_only=True)
]

model.compile(optimizer='rmsprop',
              loss='mean_squared_error',
              metrics=['MSE'])
history = model.fit(train_matrix_scaled, train_target_scaled,
                    epochs=100,
                    callbacks=callbacks_list,
                    batch_size=512,
                    validation_data=(test_matrix_scaled, test_target_scaled))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_5 (Dense)              (None, 32)                768       
_________________________________________________________________
dropout_4 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_6 (Dense)              (None, 32)                1056      
_________________________________________________________________
dropout_5 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_7 (Dense)              (None, 12)                396       
_________________________________________________________________
dropout_6 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_8 (Dense)              (None, 1)                 13        
Total para

In [50]:
model_to_load = 'full_conn_model.h5'

# Create the network and load the weights into it
model = create_fully_connected(test_matrix.shape[1])
model.load_weights(model_to_load)

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['acc'])
test_y_scaled = model.predict(test_matrix_scaled)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 32)                768       
_________________________________________________________________
dropout_7 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 32)                1056      
_________________________________________________________________
dropout_8 (Dropout)          (None, 32)                0         
_________________________________________________________________
dense_11 (Dense)             (None, 12)                396       
_________________________________________________________________
dropout_9 (Dropout)          (None, 12)                0         
_________________________________________________________________
dense_12 (Dense)             (None, 1)                 13        
Total para

In [51]:
rmse_NN = np.sqrt(mean_squared_error(test_target, scaler_target.inverse_transform(test_y_scaled)))



In [52]:
print(rmse_NN)

86.4590947179653
